# 项目3.3-神经网络压缩（网络剪枝）

## 友情提示
同学们可以前往课程作业区先行动手尝试 ！！！

## 项目描述

网络剪枝：去掉已经学习好的大model的一些权重值接近于0的分枝。

## 数据集介绍

本次使用的数据集为food-11数据集，共有11类

Bread, Dairy product, Dessert, Egg, Fried food, Meat, Noodles/Pasta, Rice, Seafood, Soup, and Vegetable/Fruit.
（面包，乳制品，甜点，鸡蛋，油炸食品，肉类，面条/意大利面，米饭，海鲜，汤，蔬菜/水果）
Training set: 9866张
Validation set: 3430张
Testing set: 3347张

**数据格式**

下载 zip 档后解压缩会有三个资料夹，分别为training、validation 以及 testing
training 以及 validation 中的照片名称格式为 [类别]_[编号].jpg，例如 3_100.jpg 即为类别 3 的照片（编号不重要）

## 项目要求

网络剪枝: 将已经学习好的大model做剪枝，让整体model变小。

## 数据准备

In [ ]:
!unzip -d work data/data58106/food-11.zip # 解压缩food-11数据集

## 环境配置/安装

无

## 简介

本项目任务是模型压缩 - Neural Network Compression。

Compression有很多种门派，在这裡我们会介绍上课出现过的其中四种，分别是:

* 知识蒸馏 Knowledge Distillation
* 网路剪枝 Network Pruning
* 用少量参数来做CNN Architecture Design
* 参数量化 Weight Quantization

在这个notebook中我们会介绍Network Pruning，
而我们有提供已经做完Knowledge Distillation的小model来做Pruning。

* Model架构 / Architecute Design在同目录中的hw7_Architecture_Design.ipynb。
* 已经train好的小model(1.5M， work/stu_net.pdparams)
  * 参数为 base=16, width_mult=1 (default)

In [ ]:
import paddle
import os
import numpy as np
import paddle.nn as nn
import paddle.optimizer as optim
import paddle.nn.functional as F
import paddle.vision.models as models

网路剪枝
===
在这裡我们会教网路剪枝
![](https://ai-studio-static-online.cdn.bcebos.com/754346f13206441aaedc32f097134cb8d479ac95e3e643338f5ab6a249f5b0e6)

简单上来说就是让一个已经学完的model中的neuron进行删减，让整个网路变得更瘦。

## Weight & Neuron 剪枝
* weight和neuron pruning差别在于prune掉一个neuron就等于是把一个matrix的整个column全部砍掉。但如此一来速度就会比较快。因为neuron pruning后matrix整体变小，但weight pruning大小不变，只是有很多空洞。

## 为什么要剪枝
* 既然要Neuron Pruning，那就必须要先衡量Neuron的重要性。衡量完所有的Neuron后，就可以把比较不重要的Neuron删减掉。
* 在这裡我们介绍一个很简单可以衡量Neuron重要性的方法 - 就是看batchnorm layer的$\gamma$因子来决定neuron的重要性。 (by paper - Network Slimming)
![](https://ai-studio-static-online.cdn.bcebos.com/6ac69ba3a4d74f1e86a02f1ed95206fceb037ff8b69d4f7b94e03fd04fd6851f)

* 相信大家看这个paddle提供的batchnorm公式应该就可以意识到为甚麽$\gamma$可以当作重要性来衡量了:)

* Network Slimming其实步骤没有这麽简单，有兴趣的同学可以check以下连结。[Netowrk Slimming](https://arxiv.org/abs/1708.06519)


## 为什么这会 work?
* 树多必有枯枝，有些neuron只是在躺分，所以有他没他没有差。
* 困难的说可以回想起老师说过的大乐透假说(The Lottery Ticket Hypothesis)就可以知道了。

## 要怎麽操作?
* 为了避免複杂的操作，我们会将StudentNet(width_mult=$\alpha$)的neuron经过筛选后移植到StudentNet(width_mult=$\beta$)。($\alpha > \beta$)
* 筛选的方法也很简单，只需要抓出每一个block的batchnorm的$\gamma$即可。

## 一些实作细节
* 假设model中间两层是这样的:

|Layer|Output # of Channels|
|-|-|
|Input|in_chs|
|Depthwise(in_chs)|in_chs|
|BatchNorm(in_chs)|in_chs|
|Pointwise(in_chs, **mid_chs**)|**mid_chs**|
|**Depthwise(mid_chs)**|**mid_chs**|
|**BatchNorm(mid_chs)**|**mid_chs**|
|Pointwise(**mid_chs**, out_chs)|out_chs|

则你会发现利用第二个BatchNorm来做筛选的时候，跟他的Neuron有直接关係的是该层的Depthwise&Pointwise以及上层的Pointwise。
因此再做neuron筛选时记得要将这四个(包括自己, bn)也要同时prune掉。

* 在Design Architecure内，model的一个block，名称所对应的Weight；

|#|name|meaning|code|weight shape|
|-|-|-|-|-|
|0|cnn.{i}.0|Depthwise Convolution Layer|nn.Conv2d(x, x, 3, 1, 1, group=x)|(x, 1, 3, 3)|
|1|cnn.{i}.1|Batch Normalization|nn.BatchNorm2d(x)|(x)|
|2||ReLU6|nn.ReLU6||
|3|cnn.{i}.3|Pointwise Convolution Layer|nn.Conv2d(x, y, 1),|(y, x, 1, 1)|
|4||MaxPooling|nn.MaxPool2d(2, 2, 0)||

In [ ]:
def network_slimming(old_model, new_model):
    params = old_model.state_dict()
    new_params = new_model.state_dict()

    # selected_idx: 每一层所选择的neuron index
    selected_idx = []
    # 我们总共有7层CNN，因此逐一抓取选择的neuron index们。
    for i in range(8):
        # 根据上表，我们要抓的gamma系数在cnn.{i}.1.weight内。
        importance = params[f'cnn.{i}.1.weight']
        # 抓取总共要筛选几个neuron。
        old_dim = len(importance)
        new_dim = len(new_params[f'cnn.{i}.1.weight'])
        # 以Ranking做Index排序，较大的会在前面(descending=True)。
        ranking = paddle.argsort(importance, descending=True)
        # 把筛选结果放入selected_idx中。
        selected_idx.append(ranking[:new_dim])

    now_processed = 1
    for (name, p1), (name2, p2) in zip(params.items(), new_params.items()):
        # 如果是cnn层，则移植参数。
        # 如果是FC层，或是该参数只有一个数字(例如batchnorm的tracenum等等资讯)，那麽就直接複製。
        if name.startswith('cnn') and p1.shape != np.array([]).shape and now_processed != len(selected_idx):
            # 当处理到Pointwise的weight时，让now_processed+1，表示该层的移植已经完成。
            if name.startswith(f'cnn.{now_processed}.3'):
                now_processed += 1

            # 如果是pointwise，weight会被上一层的pruning和下一层的pruning所影响，因此需要特判。
            if name.endswith('3.weight'):
                # 如果是最后一层cnn，则输出的neuron不需要prune掉。
                if len(selected_idx) == now_processed:
                    for idx, i in enumerate(selected_idx[now_processed - 1]):
                        temp_dim = new_params[name][idx].shape
                        new_params[name][idx].set_value(p1[int(i.numpy()[0])][:temp_dim[0]])
                # 反之，就依照上层和下层所选择的index进行移植。
                else:
                    for idx, i in enumerate(selected_idx[now_processed]):
                        temp_dim = new_params[name][idx].shape
                        new_params[name][idx].set_value(p1[int(i.numpy()[0])][:temp_dim[0]])
            else:
                for idx, i in enumerate(selected_idx[now_processed]):
                    new_params[name][idx].set_value(p1[int(i.numpy()[0])])
        else:
            new_params[name] = p1

    # 让新model load进被我们筛选过的parameters，并回传new_model。
    new_model.set_dict(new_params)
    return new_model

# 数据处理

我们的Dataset使用的是跟Hw3 - CNN同样的Dataset，因此这个区块的Augmentation / Read Image大家参考就好。

如果有不会的话可以回去看Hw3的colab。

In [ ]:
import re
import paddle
from glob import glob
from PIL import Image
import paddle.vision.transforms as transforms

class MyDataset(paddle.io.Dataset):

    def __init__(self, folderName, transform=None):
        self.transform = transform
        self.data = []
        self.label = []

        for img_path in sorted(glob(folderName + '/*.jpg')):
            try:
                # Get classIdx by parsing image path
                class_idx = int(os.path.split(img_path)[-1].split("_")[0])
            except:
                # if inference mode (there's no answer), class_idx default 0
                class_idx = 0
 
            image = Image.open(img_path)
            # Get File Descriptor
            image_fp = image.fp
            image.load()
            # Close File Descriptor (or it'll reach OPEN_MAX)
            image_fp.close()

            self.data.append(image)
            self.label.append(class_idx)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if isinstance(idx, paddle.Tensor):
            idx = idx.tolist()
        image = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, self.label[idx]


trainTransform = transforms.Compose([
    transforms.RandomCrop(256, pad_if_needed=True, padding_mode='symmetric'),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])
testTransform = transforms.Compose([
    transforms.CenterCrop(256),
    transforms.ToTensor(),
])

def get_dataloader(mode='training', batch_size=32):

    assert mode in ['training', 'testing', 'validation']

    dataset = MyDataset(
        os.path.join("/home/aistudio/work/food-11", f'{mode}'),
        transform=trainTransform if mode == 'training' else testTransform)

    dataloader = paddle.io.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=(mode == 'training'))

    return dataloader


# 预处理

 我们已经提供原始小model binary，架构是hw7_Architecture_Design.ipynb中的StudentNet。

In [ ]:
# get dataloader
train_dataloader = get_dataloader('training', batch_size=32)
valid_dataloader = get_dataloader('validation', batch_size=32)

In [ ]:
class StudentNet(paddle.nn.Layer):
    '''
      在这个网络里，我们会使用Depthwise & Pointwise Convolution Layer来做model。
      你会发现，将原本的Convolution Layer换成Dw & Pw后，Accuracy通常不会降很多。

      另外，取名为StudentNet是因为这个Model等会要做Knowledge Distillation。
    '''

    def __init__(self, base=16, width_mult=1):
        '''
          参数:
            base: 这个model一开始的通道数量，每过一层都会*2，直到base*16为止。
            width_mult: 为了之后的Network Pruning使用，在base*8 chs的Layer上会 * width_mult代表剪枝后的通道数量。
        '''
        super(StudentNet, self).__init__()
        multiplier = [1, 2, 4, 8, 16, 16, 16, 16]

        bandwidth = [base * m for m in multiplier]

        for i in range(3, 7):
            bandwidth[i] = int(bandwidth[i] * width_mult)

        self.cnn = nn.Sequential(
            nn.Sequential(

                nn.Conv2D(3, bandwidth[0], 3, 1, 1),
                nn.BatchNorm2D(bandwidth[0]),
                nn.ReLU6(),
                nn.MaxPool2D(2, 2, 0),
            ),
            nn.Sequential(
                
                nn.Conv2D(bandwidth[0], bandwidth[0], 3, 1, 1, groups=bandwidth[0]),
                nn.BatchNorm2D(bandwidth[0]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[0], bandwidth[1], 1),
                nn.MaxPool2D(2, 2, 0),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[1], bandwidth[1], 3, 1, 1, groups=bandwidth[1]),
                nn.BatchNorm2D(bandwidth[1]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[1], bandwidth[2], 1),
                nn.MaxPool2D(2, 2, 0),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[2], bandwidth[2], 3, 1, 1, groups=bandwidth[2]),
                nn.BatchNorm2D(bandwidth[2]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[2], bandwidth[3], 1),
                nn.MaxPool2D(2, 2, 0),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[3], bandwidth[3], 3, 1, 1, groups=bandwidth[3]),
                nn.BatchNorm2D(bandwidth[3]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[3], bandwidth[4], 1),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[4], bandwidth[4], 3, 1, 1, groups=bandwidth[4]),
                nn.BatchNorm2D(bandwidth[4]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[4], bandwidth[5], 1),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[5], bandwidth[5], 3, 1, 1, groups=bandwidth[5]),
                nn.BatchNorm2D(bandwidth[5]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[5], bandwidth[6], 1),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[6], bandwidth[6], 3, 1, 1, groups=bandwidth[6]),
                nn.BatchNorm2D(bandwidth[6]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[6], bandwidth[7], 1),
            ),

            nn.AdaptiveAvgPool2D((1, 1)),
        )
        self.fc = nn.Sequential(
            nn.Linear(bandwidth[7], 11),
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.reshape((out.shape[0], -1))
        return self.fc(out)

net = StudentNet()
stu_layer_state_dict = paddle.load(
    "/home/aistudio/work/stu_net.pdparams")
net.set_dict(stu_layer_state_dict)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(learning_rate=1e-3, parameters=net.parameters())

# 开始训练

* 每次Prune rate是0.95，Prune完后会重新fine-tune 3 epochs。
* 其余的步骤与你在做Hw3 - CNN的时候一样。


In [18]:
def run_epoch(dataloader, update=True, alpha=0.5):
    total_num, total_hit, total_loss = 0, 0, 0
    for now_step, batch_data in enumerate(dataloader):
        # 清空 optimizer
        optimizer.clear_gradients()
        # 处理 input
        inputs, labels = batch_data
  
        logits = net(inputs)
        loss = criterion(logits, labels)
        if update:
            loss.backward()
            optimizer.step()
        
        total_hit += (paddle.argmax(logits, axis=-1).numpy() == labels.numpy).sum()
        total_num += len(inputs)
        total_loss += loss.numpy()[0] * len(inputs)

    return total_loss / total_num, total_hit / total_num

now_width_mult = 1
for i in range(5):
    now_width_mult *= 0.95
    new_net = StudentNet(width_mult=now_width_mult)
    params = net.state_dict()
    net = network_slimming(net, new_net)
    now_best_acc = 0
    for epoch in range(5):
        net.train()
        train_loss, train_acc = run_epoch(train_dataloader, update=True)
        net.eval()
        valid_loss, valid_acc = run_epoch(valid_dataloader, update=False)
        # 在每个width_mult的情况下，存下最好的model。
        if valid_acc > now_best_acc:
            now_best_acc = valid_acc
            paddle.save(net.state_dict(), f'custom_small_rate_{now_width_mult}.pdparams')
        print('rate {:6.4f} epoch {:>3d}: train loss: {:6.4f}, acc {:6.4f} valid loss: {:6.4f}, acc {:6.4f}'.format(now_width_mult, 
            epoch, train_loss, train_acc, valid_loss, valid_acc))
